In [1]:
from joblib import load
from os.path import join
# import argparse
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


from mvmm_sim.simulation.sim_viz import save_fig
from mvmm_sim.data_analysis.multi_view.viz_resuls import plot_Pi
from mvmm.multi_view.block_diag.graph.bipt_community import community_summary
from mvmm_sim.data_analysis.multi_view.viz_resuls import plot_Pi

from explore.CatCat import CatCat

inches = 15
dpi = 200

plt.rcParams["axes.labelsize"] = 25
sns.set_style("whitegrid")

save_dir = 'mouse_et_figures'

In [2]:
fpaths=['/Users/iaincarmichael/Dropbox/Research/mvmm/public_release/mouse_et/pro_data/transcriptomic_select_markers_pca_feats.csv',
        '/Users/iaincarmichael/Dropbox/Research/mvmm/public_release/mouse_et/pro_data/ephys_pca_feats.csv']
results_dir='' # set the results directory

vars2compare_fpath='/Users/iaincarmichael/Dropbox/Research/mvmm/public_release/mouse_et/pro_data/vars2compare.csv'
metadata_fpath = '/Users/iaincarmichael/Dropbox/Research/mvmm/public_release/mouse_et/pro_data/metadata.csv'

interp_dir = join(results_dir, 'interpret', 'bd_mvmm')



In [3]:
vars2compare = pd.read_csv(vars2compare_fpath, index_col=0)

# map super types to sub types
super2sub = vars2compare.groupby('transcr_super_type')['transcr_subtype'].unique()

pi_data = load(join(interp_dir, 'pi_data'))
bd_mvmm = load(join(results_dir, 'model_fitting', 'selected_models'))['bd_mvmm'].final_


y_block_pred = pd.read_csv(join(results_dir, 'interpret', 'bd_mvmm', 'block', 'y_pred_block.csv'), header=None, index_col=0)
y_block_pred.columns = ['block']
y_block_pred.index.name = 'specimen_id'


# Pi matrix

In [4]:
D = pi_data['Pi_block_perm']
D.index.name = "RNA clusters"
D.columns.name = "Ephys clusters"


plt.figure(figsize=(inches, inches))
plot_Pi(D, mask=pi_data['Pi_block_perm_zero_mask'], cmap="Blues", cbar=True)
save_fig(join(save_dir, 'mouse_et_bd_pi.png'), dpi=dpi)

# subtypes

In [81]:
# md = load(join(results_dir, 'interpret', 'bd_mvmm', 'block', 'metadata_comparisons_block'))
# cross_counts = md.comparisons_.iloc[0,0].cross_

md = CatCat().fit(y_block_pred['block'].astype(str), vars2compare['transcr_subtype'])
cross_counts = md.cross_

cross_counts.columns = list(cross_counts.columns) # gets rid of annoying categorical index

In [109]:
# all_cols = np.array(cross_counts_14.columns.values)
all_cols = cross_counts.columns
cross_counts_14 = cross_counts.loc[[str(1), str(4)]]
cols2keep = all_cols[cross_counts_14.sum() > 0]

other_cols = list(set(all_cols).difference(cols2keep))

results_14 = cross_counts[cols2keep]
results_14['Other subtypes'] = cross_counts[other_cols].sum(axis=1).values


/Users/iaincarmichael/anaconda3/envs/mvmm_public/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [113]:
results_14.index.name = 'MVMM Block'
results_14.columns.name = 'Transcriptomic Subtype'

labels = results_14.values.astype(int).astype(str).copy()

# change this very large value so the heamtmaps looks nicer 
results_14.loc[str(2), 'Other subtypes'] = 120 
results_14[results_14 == 0] = np.nan


plt.figure(figsize=(4, 8))
# sns.heatmap(results_14.T, vmin=0, cmap='Blues',
#             linewidths=.5,
#             annot=True, fmt='1.0f', cbar=False)

sns.heatmap(results_14.T, vmin=0, cmap='Blues',
            linewidths=.5,
            annot=labels.T,
            cbar=False, fmt='')

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

save_fig(join(save_dir, 'block_vs_transcr_subtype.png'), dpi=dpi)

/Users/iaincarmichael/anaconda3/envs/mvmm_public/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/iaincarmichael/anaconda3/envs/mvmm_public/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# ephys curves

In [4]:
from mvmm_sim.data_analysis.utils import load_data
from mvmm_sim.mouse_et.MouseETPaths import MouseETPaths
from mvmm_sim.mouse_et.raw_ephys_loading import load_raw_ephys
from mvmm_sim.mouse_et.ephys_viz import get_ephys_super_data,\
    plot_top_clust_ephys_curves, plot_cluster_ephys_curve

In [5]:

fitting_dir = join(results_dir, 'model_fitting')
ephys_viz_dir = join(results_dir, 'interpret', 'bd_mvmm', 'ephys_pca_feats')


# load models and data
models = load(join(fitting_dir, 'selected_models'))
view_data, dataset_names, sample_names, view_feat_names = load_data(*fpaths)


# load raw ephys data
orig_data_dir = join(MouseETPaths().raw_data_dir, 'inh_patchseq_spca_files',
                     'orig_data_csv')
ephys_raw = load_raw_ephys(orig_data_dir, concat=False)
for k in ephys_raw.keys():
    ephys_raw[k] = ephys_raw[k].loc[sample_names]
    print(k, ephys_raw[k].shape)
n_datasets = len(ephys_raw)


# get data for plotting
v = 1
cluster_super_means, super_data_means, super_data_stds, y_cnts = \
    get_ephys_super_data(model=models['bd_mvmm'].final_.view_models_[v],
                         fit_data=view_data[v],
                         ephys_raw=ephys_raw)

clust_labels = ['cluster_{}'.format(cl_idx + 1)
                for cl_idx in range(len(y_cnts))]


inst_freq_data (4269, 150)
spiking_threshold_v_data (4269, 150)
first_ap_v_data (4269, 300)
spiking_width_data (4269, 150)
first_ap_dv_data (4269, 298)
spiking_peak_v_data (4269, 150)
spiking_upstroke_downstroke_ratio_data (4269, 150)
isi_shape_data (4269, 100)
step_subthresh_data (4269, 700)
subthresh_norm_data (4269, 140)
spiking_fast_trough_v_data (4269, 150)
inst_freq_norm_data (4269, 150)


In [37]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter

from mvmm_sim.viz_utils import simpleaxis
from mvmm_sim.data_analysis.super_means import get_super_means

from mvmm_sim.mouse_et.ephys_viz import plot_ephys_curve

from matplotlib import pyplot as plt
plt.rcParams["axes.grid"] = False
plt.rcParams["axes.labelsize"] = 50


def transform_name(name):
    return ' '.join(name.split('_')[:-1])


In [52]:
cluster_super_means['inst_freq_data']

array([[ 43.48214638,  34.21645609,  33.1578873 , ..., 118.93180008,
        119.90877415, 119.19885043],
       [ 21.6860223 ,  15.07716158,  11.6222502 , ...,  18.09862479,
         17.8860368 ,  17.48738863],
       [ 21.91498577,   9.04939444,   4.88851235, ...,   1.35861146,
          1.34273167,   1.3268519 ],
       ...,
       [ 38.89848592,  32.66226753,  29.14623409, ...,  66.06592199,
         65.25248499,  64.54163688],
       [ 12.17804854,   9.32094899,   6.81688965, ...,   9.9728572 ,
          9.96879539,   9.86985401],
       [  2.45771631,   2.43341667,   2.40911702, ...,  20.91629232,
         20.87570233,  20.86073157]])

In [53]:
cl_idx = 36 - 1
to_show = ['first_ap_v_data', 'spiking_width_data', 'spiking_peak_v_data',
           'spiking_upstroke_downstroke_ratio_data', 'isi_shape_data',
           'step_subthresh_data', 'inst_freq_norm_data']


label = clust_labels[cl_idx]

values = {}
for name in cluster_super_means.keys():
    values[name] = cluster_super_means[name][cl_idx]

# values = {k: values[k] for k in to_show}



# plt.figure(figsize=(2 * len(values) * inches, inches))
# plot_cluster_ephys_curve(values,
#                          overall_means=super_data_means,
#                          overall_stds=super_data_stds,
#                          y_label=label)

overall_means=super_data_means
overall_stds=super_data_stds
y_label=label

In [54]:
n_data_types = len(values)
max_n_cols = 4
n_cols = min(max_n_cols, n_data_types)
n_rows = np.ceil(n_data_types / max_n_cols).astype(int)

plt.figure(figsize=(inches * n_cols, inches * n_rows))
grid = plt.GridSpec(nrows=n_rows, ncols=n_cols,
                    wspace=0.4)  #, hspace=0.4)


for data_idx, data_name in enumerate(values.keys()):
    if overall_means is not None:
        m = overall_means[data_name]
    if overall_stds is not None:
        s = overall_stds[data_name]


    col_idx = data_idx % n_cols
    row_idx = data_idx // n_cols
    
    plt.subplot(grid[row_idx, col_idx])
    
    plot_ephys_curve(values[data_name], overall_mean=m, overall_std=s,
                         color='red')
    plt.ylabel(transform_name(data_name))
    plt.xlabel('Time')
    

save_fig(join(save_dir, 'ephys_curve_cl_{}.png'.format(cl_idx + 1)), dpi=dpi)